# Datasets
Author: Javier Duarte


## Converting datasets

In [7]:
%%bash
root2hdf5 -f data/ntuple_4mu_gg.root data/ntuple_4mu_VV.root

INFO:rootpy.root2hdf5] Converting data/ntuple_4mu_gg.root ...
INFO:rootpy.root2hdf5] Will convert 1 tree in /
INFO:rootpy.root2hdf5] Converting tree 'HZZ4LeptonsAnalysisReduced' with 22344 entries ...
INFO:rootpy.root2hdf5] Created data/ntuple_4mu_gg.h5
INFO:rootpy.root2hdf5] Converting data/ntuple_4mu_VV.root ...
INFO:rootpy.root2hdf5] Will convert 1 tree in /
INFO:rootpy.root2hdf5] Converting tree 'HZZ4LeptonsAnalysisReduced' with 25817 entries ...
INFO:rootpy.root2hdf5] Created data/ntuple_4mu_VV.h5


In [8]:
import numpy as np
import pandas as pd
import h5py

treename = 'HZZ4LeptonsAnalysisReduced'
VARS = ['f_massjj','f_deltajj']
filename = {}
h5file = {}
params = {}
df = {}

filename['VV'] = 'data/ntuple_4mu_VV.h5'
filename['gg'] = 'data/ntuple_4mu_gg.h5'

h5file['VV'] = h5py.File(filename['VV'], 'r') # open read-only
params['VV'] = h5file['VV'][treename][()]
h5file['gg'] = h5py.File(filename['gg'], 'r') # open read-only
params['gg'] = h5file['gg'][treename][()]

/cvmfs/cms.cern.ch/slc7_amd64_gcc700/external/py2-pippkgs_depscipy/3.0-omkpbe4/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
print params['VV'].dtype.names
print ''

df['VV'] = pd.DataFrame(params['VV'],columns=VARS)
df['gg'] = pd.DataFrame(params['gg'],columns=VARS)

df['VV']= df['VV'][(df['VV'].f_massjj > 0) & (df['VV'].f_deltajj > 0)] # cut out undefined variables
df['gg']= df['gg'][(df['gg'].f_massjj > 0) & (df['gg'].f_deltajj > 0)] # cut out undefined variables

df['VV']['isSignal'] = np.ones(len(df['VV'])) # add isSignal variable
df['gg']['isSignal'] = np.zeros(len(df['gg'])) # add isSignal variable

print df['VV'].iloc[:5] # print first 5 entries
print ''
print df['gg'].iloc[:5] # print first 5 entries

('f_run', 'f_lumi', 'f_event', 'f_weight', 'f_int_weight', 'f_pu_weight', 'f_eff_weight', 'f_lept1_pt', 'f_lept1_eta', 'f_lept1_phi', 'f_lept1_charge', 'f_lept1_pfx', 'f_lept1_sip', 'f_lept2_pt', 'f_lept2_eta', 'f_lept2_phi', 'f_lept2_charge', 'f_lept2_pfx', 'f_lept2_sip', 'f_lept3_pt', 'f_lept3_eta', 'f_lept3_phi', 'f_lept3_charge', 'f_lept3_pfx', 'f_lept3_sip', 'f_lept4_pt', 'f_lept4_eta', 'f_lept4_phi', 'f_lept4_charge', 'f_lept4_pfx', 'f_lept4_sip', 'f_iso_max', 'f_sip_max', 'f_Z1mass', 'f_Z2mass', 'f_angle_costhetastar', 'f_angle_costheta1', 'f_angle_costheta2', 'f_angle_phi', 'f_angle_phistar1', 'f_pt4l', 'f_eta4l', 'f_mass4l', 'f_mass4lErr', 'f_njets_pass', 'f_deltajj', 'f_massjj', 'f_D_jet', 'f_jet1_pt', 'f_jet1_eta', 'f_jet1_phi', 'f_jet1_e', 'f_jet2_pt', 'f_jet2_eta', 'f_jet2_phi', 'f_jet2_e', 'f_D_bkg_kin', 'f_D_bkg', 'f_D_gg', 'f_D_g4', 'f_Djet_VAJHU', 'f_pfmet')

      f_massjj  f_deltajj  isSignal
0  1300.426880   5.128572       1.0
1   437.221863   3.278692       1.0
3  